In [1]:
import os

list_file = sorted(list(os.walk('./6.menu_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['menu_2-serial_13.xlsx', 'menu_2-serial_27.xlsx', 'menu_2-serial_43.xlsx']

In [2]:
file = list_file[0]

file

'menu_2-serial_13.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./6.menu_2/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：1


['354099367853']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200 or resp.status_code == 404:
            break
    except:
        continue

resp

1


<Response [200]>

In [8]:
resp.status_code == 404

False

In [9]:
import json

json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

json_

{'o': {'l': 1,
  'g': {'enableATCRedesign': 'false',
   'lazyLoaderType': False,
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'},
      'dropTag': 'server'},
     'enableSMEMerchBTF': {'value': False},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'true',
       'DELI_ENABLED': 'false'}},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'BXR_BUNDLED_TEST': {'value': {'ADS_SME_BTF_ENABLED': 'true',
       'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'}},
     'lazyLoaderType': {'value': False}},
    'qTags': {}},
   'dWebATFSplit': 'DEFAULT',
   'renderProgressively': True,
   'isPrefetchRe

In [10]:
if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'

print(market, country, language, locale)

json_['o']['g']['marketplaceId'].strip() != market or json_['o']['g']['country'].strip() != country or json_['o']['g']['language'].strip() != language or json_['o']['g']['locale'].strip() != locale or json_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']

EBAY-US US en en-US


False

In [11]:
index_model = -1
for i in range(len(json_['o']['w'])):
    if 'model' in json_['o']['w'][i][2]:
        index_model = i
        break

index_model

0

In [12]:
index_model == -1

False

In [13]:
compatibility = json_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in json_['o']['w'][index_model][2]['model']['modules'] else ''
dict_compatibility = {} if compatibility == '' else {'api': compatibility['action']['URL'],
                                                     'page': int(compatibility['text'].strip()),
                                                     'data': {'scopedContext': {'catalogDetails': {'categoryId': compatibility['action']['params']['categoryId'],
                                                                                                   'itemId': compatibility['action']['params']['itemId'],
                                                                                                   'marketplaceId': compatibility['action']['params']['marketplaceId'],
                                                                                                   'sellerName': compatibility['action']['params']['sellerName']}}}}

dict_compatibility

{'api': 'https://www.ebay.com/g/api/finders',
 'page': 1,
 'data': {'scopedContext': {'catalogDetails': {'categoryId': '33640',
    'itemId': '354099367853',
    'marketplaceId': 'EBAY-US',
    'sellerName': 'carpartswholesale'}}}}

In [14]:
dict_specific = {}
dict_item = json_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']
for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{'0': {'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details."},
 '1': {'Anticipated Ship Out Time': 'Same day - 1 business day'},
 '2': {'Brand': 'Perfect Fit'},
 '3': {'Fitment Type': 'Direct Replacement'},
 '4': {'Interchange Part Number': '2.0L, 2018 2017 2016 2015, 2058850637, 4Cyl, Base 4Matic, Bumper Absorber, C300, Coupe, For Models Without AMG Styling Package and Distronic, Cruise Control, Convertible, For Models Without AMG Styling, Front, Center, MB1070112, Mercedes-Benz, Package and Distronic Cruise Control, Sedan Coupe Convertible, Sedan, For Models Without AMG Styling Package and Luxury Package'},
 '5': {'Manufacturer Part Number': '14865775'},
 '6': {'Manufacturer Warranty': '1-year u

In [15]:
description_url = json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

description_url

'https://vi.vipr.ebaydesc.com/itmdesc/354099367853?t=1715346595000&category=33640&seller=carpartswholesale&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1'

In [16]:
title = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

'Front Center Bumper Absorber For 2015-18 Mercedes Benz C300 2058850637 MB1070112'

In [17]:
subtitle = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

subtitle

'Hassle Free Returns. Free Fast Shipping.'

In [18]:
price = json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
# price = json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

'US $88.89'

In [19]:
available, sold = '', ''
list_quantity = json_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][index_model][2]['model']['modules'] else []
for quantity in list_quantity:
    if input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
        if 'verfügbar' in quantity['text']:
            available = quantity['text'].strip()
        elif 'verkauft' in quantity['text']:
            sold = quantity['text'].strip()
    else:
        if 'available' in quantity['text']:
            available = quantity['text'].strip()
        elif 'sold' in quantity['text']:
            sold = quantity['text'].strip()

available, sold

('More than 10 available', '33 sold')

In [20]:
dict_src = {}
list_media = json_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in json_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
for i, media in zip(range(len(list_media)), list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

{'0': 'https://i.ebayimg.com/images/g/sK4AAOSwVFZmPhzo/s-l500.jpg',
 '1': 'https://i.ebayimg.com/images/g/y~MAAOSwNldmPhzo/s-l500.jpg',
 '2': 'https://i.ebayimg.com/images/g/ZDUAAOSwg7dmPhzo/s-l500.jpg',
 '3': 'https://i.ebayimg.com/images/g/bH0AAOSwJklmPhzo/s-l500.jpg',
 '4': 'https://i.ebayimg.com/images/g/IH0AAOSwC29mPhzo/s-l500.jpg'}

In [21]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Title': title,
                         'Subtitle': subtitle,
                         'Price': price,
                         'Available': available,
                         'Sold': sold,
                         'Url': input_.loc[a, 'Url'],
                         'Description Url': description_url,
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specific': json.dumps(dict_specific),
                         'Json_Compatibility': json.dumps(dict_compatibility)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Title,Subtitle,Price,Available,Sold,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,37131,354099367853,Front Center Bumper Absorber For 2015-18 Merce...,Hassle Free Returns. Free Fast Shipping.,US $88.89,More than 10 available,33 sold,https://www.ebay.com/itm/354099367853?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/354099367...,"{""0"": ""https://i.ebayimg.com/images/g/sK4AAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,...","{""api"": ""https://www.ebay.com/g/api/finders"", ..."


In [22]:
crawler_status = 'ok'

crawler_status

'ok'

In [23]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                             'Item Number': input_.loc[a, 'Item Number'],
                             'Url': input_.loc[a, 'Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [24]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_item_1.xlsx', index=False)

output_correct

,No,Item Number,Title,Subtitle,Price,Available,Sold,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,37131,354099367853,Front Center Bumper Absorber For 2015-18 Merce...,Hassle Free Returns. Free Fast Shipping.,US $88.89,More than 10 available,33 sold,https://www.ebay.com/itm/354099367853?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/354099367...,"{""0"": ""https://i.ebayimg.com/images/g/sK4AAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,...","{""api"": ""https://www.ebay.com/g/api/finders"", ..."


In [25]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_item_1_error.xlsx', index=False)

output_error

""
